# Building a news recommendation system

The goal of this exercise is to train and serve a news recommendation system.

### Downloading and preparing the training data

First we will download and extract a subset of hackernews articles. This is a small subset of the articles that have been submitted to https://news.ycombinator.com. 

In [ ]:
!wget https://s3-us-west-2.amazonaws.com/ray-tutorials/hackernews.zip

In [ ]:
!unzip -o hackernews.zip

In [ ]:
import ray
ray.init()

The following is a function to parse a chunk of the data and produce a pandas DataFrame with the title and the score of the submissions.

In [ ]:
import json
import pandas as pd
import numpy as np

@ray.remote
def parse_hn_submissions(path):
    with open(path, "r") as f:
        records = []
        for line in f.readlines():
            body = json.loads(line)["body"]
            records.append({"data": body["title"], "score": body["score"]})
        return pd.DataFrame(records)

In [ ]:
files = ["hackernews-" + str(i) + ".json" for i in range(1, 5)]
# %time records = [parse_hn_submissions(file) for file in files]

In [ ]:
%time results = ray.get([parse_hn_submissions.remote(file) for file in files])
df = pd.concat(results)

In [ ]:
df["score"].mean()

In [ ]:
df["target"] = df["score"] > 9.0

### Training a model

Now we split the data into a train and test set.

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.2)

The following defines a pipeline that first converts the title of the submission to a bag of words and then applies an SVM for classification.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import SGDClassifier

pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('clf', SGDClassifier(loss='hinge', penalty='l2',
                          alpha=1e-3, random_state=42,
                          max_iter=5, tol=None))])

In [ ]:
result = pipeline.fit(train.data, train.target)

In [ ]:
predicted = pipeline.predict(test.data)

In [ ]:
np.mean(predicted == test.target)

### Hyperparameter tuning

In [ ]:
from ray import tune

In [ ]:
def train_func(config, reporter):
    pipeline = Pipeline([
        ('vect', CountVectorizer()),
        ('clf', SGDClassifier(loss='hinge', penalty='l2',
                              alpha=config["alpha"], random_state=42,
                              max_iter=5, tol=None))])

    pipeline.fit(train.data, train.target)
    reporter(mean_accuracy=np.mean(pipeline.predict(train.data) == train.target)) # report metrics

In [ ]:
all_trials = tune.run(
    train_func,
    name="quick-start",
    stop={"mean_accuracy": 99},
    config={"alpha": tune.grid_search([1e-1, 1e-3, 1e-5])}
)